# <center> Invoice Reading Automation :
## <center> OCR & Text Processing 
    
    
Ce code est un toymodel pour illustrer des idées d'automatisation avec une intervention humaine dans le process. On s'intéresse ici au process de lecture et extraction d'informations de factures. Ce process est un process F&A. 
    

In [34]:
from io import BytesIO
import pytesseract
from pdf2image import convert_from_bytes
import ipywidgets as widgets
from IPython.display import display
import re

## <center> Functions and Codes

In [35]:
# Fonction pour extraire le texte à partir d'une image
def extract_text_from_image(image):
    text = pytesseract.image_to_string(image)
    return text

def extract_prices(text,symbol='$'):
    pattern = r"(\d{1,3}(?:[.,]\d{3})*(?:,\d+)?)\s*€"

    matches = re.findall(pattern, text)
    #print('matches=',matches)
    # Afficher les prix extraits
    #for match in matches:
     #   print(match)
    return matches

# Fonction pour traiter le fichier PDF
def process_pdf(file_content):
    # Convertir le contenu du fichier PDF en images
    images = convert_from_bytes(file_content)
    
    extracted_text = ""
    
    # Extraire le texte de chaque image
    for image in images:
        text = extract_text_from_image(image)
        extracted_text += text
    #print('extracted_text=',extracted_text)
    symbol='$'
    extracted_prices=extract_prices(extracted_text,symbol)
   # print('extract_prices=',extracted_prices)
            # Convertir les éléments de la liste en nombres décimaux
    nombres = [float(element.replace(',', '.')) for element in extracted_prices]

    # Trouver le plus grand nombre dans la liste
    total = str(max(nombres))
    
    return extracted_text,total+symbol

# Fonction de gestion de l'upload du fichier
def handle_upload(change):
    uploaded_file = list(change['new'].values())[0]
    
    # Lire le contenu du fichier
    content = uploaded_file['content']
    
    # Traiter le fichier PDF
    extracted_text, total_text = process_pdf(content)
    #print('total=',total_text)
    
    #print('extracted_text=',extracted_text.find)
    
    # Afficher le texte extrait
    output1_text.value = extracted_text
    #print('output_text.value=',output_text.value)
    
    # Afficher le texte extrait
   # print('total_text=',total_text)
    output2_text.value = total_text




## <center> User Interface

In [36]:
# Créer un widget de téléchargement de fichier
upload_button = widgets.FileUpload(description='Uploader un fichier')

# Créer un widget pour afficher le texte extrait
output1_text = widgets.Textarea(description='Texte extrait', disabled=True,layout=widgets.Layout(height="100%", width="auto"))

# Créer un widget pour afficher le total de la facture.
output2_text = widgets.Textarea(description='Total de la facture', disabled=True,layout=widgets.Layout(height="50%", width="50%"))

# Associer la fonction de gestion de l'upload au widget de téléchargement de fichier
upload_button.observe(handle_upload, names='value')

# Afficher les widgets
display(upload_button)
display(output1_text)
display(output2_text)

FileUpload(value={}, description='Uploader un fichier')

Textarea(value='', description='Texte extrait', disabled=True, layout=Layout(height='100%', width='auto'))

Textarea(value='', description='Total de la facture', disabled=True, layout=Layout(height='50%', width='50%'))

In [30]:
import re
from difflib import SequenceMatcher
import ipywidgets as widgets
from IPython.display import display
from io import BytesIO
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage

# Variable pour stocker le texte extrait du PDF
extracted_text = None

# Fonction pour extraire le texte du PDF
def extract_text_from_pdf(file_content):
    resource_manager = PDFResourceManager()
    output_string = BytesIO()
    layout_params = LAParams()

    with TextConverter(resource_manager, output_string, codec='utf-8', laparams=layout_params) as converter:
        interpreter = PDFPageInterpreter(resource_manager, converter)
        for page in PDFPage.get_pages(BytesIO(file_content)):
            interpreter.process_page(page)

    extracted_text = output_string.getvalue().decode('utf-8', 'ignore')
    output_string.close()

    return extracted_text

# Fonction pour trouver le motif regex le plus proche
def find_closest_regex_match(examples, extracted_text):
    closest_match = None
    closest_ratio = 0

    for regex in re.findall(r'\b\w+\b', extracted_text):
        for example in examples:
            ratio = SequenceMatcher(None, example, regex).ratio()
            if ratio > closest_ratio:
                closest_ratio = ratio
                closest_match = regex

    return closest_match

# Fonction de gestion du clic sur le bouton
def handle_button_click(button):
    dialog = widgets.Textarea(value='', placeholder='Saisir une ou plusieurs exemples', description='Exemples:')
    display(dialog)

    def handle_dialog_submit(button):
        examples = dialog.value.split('\n')

        # Trouver le motif regex le plus proche parmi les exemples dans le texte extrait
        closest_match = find_closest_regex_match(examples, extracted_text)

        if closest_match:
            print("Motif regex le plus proche:")
            print(closest_match)
        else:
            print("Aucun motif regex trouvé.")

    submit_button = widgets.Button(description='Valider')
    submit_button.on_click(handle_dialog_submit)
    display(submit_button)

# Fonction de gestion de l'upload du fichier
def handle_upload(change):
    uploaded_file = list(change['new'].values())[0]
    
    # Lire le contenu du fichier PDF
    global file_content
    file_content = uploaded_file['content']
    
    # Extraire le texte du PDF
    global extracted_text
    extracted_text = extract_text_from_pdf(file_content)
    
    # Afficher le texte extrait du PDF une seule fois
    print("Texte extrait du PDF :")
    print(extracted_text)
    
    # Créer le bouton d'action
    action_button = widgets.Button(description='Rechercher un motif regex')
    action_button.on_click(handle_button_click)
    display(action_button)

# Créer un widget de téléchargement de fichier
upload_button = widgets.FileUpload(description='Uploader un fichier')

# Associer la fonction de gestion de l'upload au widget de téléchargement de fichier
upload_button.observe(handle_upload, names='value')

# Afficher le widget de téléchargement de fichier
display(upload_button)



FileUpload(value={}, description='Uploader un fichier')

Texte extrait du PDF :
Voici le reçu de votre course, Wiem

Nous espérons que vous avez apprécié votre course ce matin.

Total

Prix de la course

Sous-total

Contribution pour la Mobilité Electrique

Paiements

Espèces

25/06/2023 17:00

25 juin 2023

46,42 €

45,77 €

45,77 €

0,65 €

46,42 €

Pour consulter vos reçus (et vos factures le cas échéant), ou pour obtenir plus d'informations, rendez-vous sur votre espace personnel.

Votre chauffeur était Marc

Reçu émis au nom de BLACK CAR

UberX     20.91 kilomètres | 1 h 12 min

15:46 | 9 Rue la Vieuville, 75018 Paris, France

17:00 | Avenue de l'Union & Rue de Cardiff, Paray-Vieille-Poste, Île-de-France, 94390

Le prix n'inclut pas d'éventuels frais bancaires. Contactez votre banque pour en savoir plus sur les frais applicables.




Button(description='Rechercher un motif regex', style=ButtonStyle())

In [29]:
import re
from difflib import SequenceMatcher
import ipywidgets as widgets
from IPython.display import display
from io import BytesIO
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage

# Variable pour stocker le texte extrait du PDF
extracted_text = None

# Fonction pour extraire le texte du PDF
def extract_text_from_pdf(file_content):
    resource_manager = PDFResourceManager()
    output_string = BytesIO()
    layout_params = LAParams()

    with TextConverter(resource_manager, output_string, codec='utf-8', laparams=layout_params) as converter:
        interpreter = PDFPageInterpreter(resource_manager, converter)
        for page in PDFPage.get_pages(BytesIO(file_content)):
            interpreter.process_page(page)

    extracted_text = output_string.getvalue().decode('utf-8', 'ignore')
    output_string.close()

    return extracted_text

# Fonction pour trouver le motif regex le plus simple
def find_simplest_regex(examples):
    regex_parts = []
    for example in examples:
        regex_part = ''
        for char in example:
            if char.isalpha():
                regex_part += r'\w'
            elif char.isdigit():
                regex_part += r'\d'
            else:
                regex_part += re.escape(char)
        regex_parts.append(regex_part)
    regex = r'^(' + '|'.join(regex_parts) + r')$'
    return regex

# Fonction pour trouver le mot le plus proche du regex dans le texte extrait
def find_closest_match(regex, extracted_text):
    matches = re.findall(regex, extracted_text)
    if matches:
        closest_match = max(matches, key=lambda match: SequenceMatcher(None, match, extracted_text).ratio())
        return closest_match
    return None

# Fonction de gestion du clic sur le bouton
def handle_button_click(button):
    dialog = widgets.Textarea(value='', placeholder='Saisir une ou plusieurs exemples', description='Exemples:')
    display(dialog)

    def handle_dialog_submit(button):
        examples = dialog.value.split('\n')

        # Trouver le motif regex le plus simple
        regex = find_simplest_regex(examples)

        # Trouver le mot le plus proche du regex dans le texte extrait
        closest_match = find_closest_match(regex, extracted_text)

        if closest_match:
            print("Mot le plus proche du regex dans le texte extrait:")
            print(closest_match)
        else:
            print("Aucun mot trouvé.")

    submit_button = widgets.Button(description='Valider')
    submit_button.on_click(handle_dialog_submit)
    display(submit_button)

# Fonction de gestion de l'upload du fichier
def handle_upload(change):
    uploaded_file = list(change['new'].values())[0]
    
    # Lire le contenu du fichier PDF
    global file_content
    file_content = uploaded_file['content']
    
    # Extraire le texte du PDF
    global extracted_text
    extracted_text = extract_text_from_pdf(file_content)
    
    # Afficher le texte extrait du PDF une seule fois
    print("Texte extrait du PDF :")
    print(extracted_text)
    
    # Créer le bouton d'action
    action_button = widgets.Button(description='Rechercher le mot le plus proche du regex')
    action_button.on_click(handle_button_click)
    display(action_button)

# Créer un widget de téléchargement de fichier
upload_button = widgets.FileUpload(description='Uploader un fichier')

# Associer la fonction de gestion de l'upload au widget de téléchargement de fichier
upload_button.observe(handle_upload, names='value')

# Afficher le widget de téléchargement de fichier
display(upload_button)



FileUpload(value={}, description='Uploader un fichier')

Texte extrait du PDF :
Voici le reçu de votre course, Wiem

Nous espérons que vous avez apprécié votre course ce matin.

Total

Prix de la course

Sous-total

Contribution pour la Mobilité Electrique

Paiements

Espèces

25/06/2023 17:00

25 juin 2023

46,42 €

45,77 €

45,77 €

0,65 €

46,42 €

Pour consulter vos reçus (et vos factures le cas échéant), ou pour obtenir plus d'informations, rendez-vous sur votre espace personnel.

Votre chauffeur était Marc

Reçu émis au nom de BLACK CAR

UberX     20.91 kilomètres | 1 h 12 min

15:46 | 9 Rue la Vieuville, 75018 Paris, France

17:00 | Avenue de l'Union & Rue de Cardiff, Paray-Vieille-Poste, Île-de-France, 94390

Le prix n'inclut pas d'éventuels frais bancaires. Contactez votre banque pour en savoir plus sur les frais applicables.




Button(description='Rechercher le mot le plus proche du regex', style=ButtonStyle())

In [ ]:
print(output_text.value)

In [ ]:
import pandas as pd
from io import BytesIO
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
import ipywidgets as widgets
from IPython.display import display


In [ ]:

# Fonction pour extraire le texte du PDF
def extract_text_from_pdf(file_content):
    resource_manager = PDFResourceManager()
    output_string = BytesIO()
    layout_params = LAParams()

    with TextConverter(resource_manager, output_string, codec='utf-8', laparams=layout_params) as converter:
        interpreter = PDFPageInterpreter(resource_manager, converter)
        for page in PDFPage.get_pages(BytesIO(file_content)):
            interpreter.process_page(page)

    extracted_text = output_string.getvalue().decode('utf-8', 'ignore')
    output_string.close()

    return extracted_text




# Fonction pour convertir le texte du tableau en dataframe
def convert_table_to_dataframe(table_text):
    table_lines = table_text.split('\n')
    table_data = [line.split('\t') for line in table_lines if line.strip() != '']
    df = pd.DataFrame(table_data[1:], columns=table_data[0])
    return df

# Fonction de gestion de l'upload du fichier
def handle_upload(change):
    uploaded_file = list(change['new'].values())[0]
    
    # Lire le contenu du fichier PDF
    content = uploaded_file['content']
    
    # Extraire le texte du PDF
    pdf_text = extract_text_from_pdf(content)
    
    # Convertir le texte du tableau en dataframe
    df = convert_table_to_dataframe(pdf_text)
    
    # Enregistrer le dataframe au format Excel
    excel_output = BytesIO()
    with pd.ExcelWriter(excel_output, engine='xlsxwriter') as writer:
        df.to_excel(writer, index=False, sheet_name='Sheet1')
    excel_output.seek(0)
    
    # Afficher le lien de téléchargement du fichier Excel
    output_link.value = f'<a href="data:application/vnd.openxmlformats-officedocument.spreadsheetml.sheet;base64,{excel_output.getvalue().decode()}">Télécharger le fichier Excel</a>'

# Créer un widget de téléchargement de fichier
upload_button = widgets.FileUpload(description='Uploader un fichier')

# Créer un widget pour afficher le lien de téléchargement
output_link = widgets.HTML()

# Associer la fonction de gestion de l'upload au widget de téléchargement de fichier
upload_button.observe(handle_upload, names='value')

# Afficher les widgets
display(upload_button)
display(output_link)


In [ ]:
int('f')